In [1]:
import argparse, os
import cv2
import numpy as np
import imageio
import time

In [2]:
def readPicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)#cv2.IMREAD_UNCHANGED is important that the output is (x,y,ChannelRGB)
    print(img.shape)
    #alternative
    #img=imageio.imread(picturepath) #liest Bild von picturepath
    return(img)

In [3]:
yuv = readPicture('/home/nico/rohdaten/klein/wasteland_clouds_16k.hdr_537.hdr')
yuv = YUVtoRGB(yuv)
savePic(yuv,'LuPatest_6','hdr','/home/nico/rohdaten/klein/')

(320, 320, 3)


NameError: name 'YUVtoRGB' is not defined

In [4]:
def savePic(picture,fileName,extention,outPath):
    outPath = outPath+fileName+'.'+extention # combines the path with the name and extention of the file
    print(outPath)
    try:
        #imageio.imwrite(outPath,picture,format=extention)# old way
        cv2.imwrite(outPath,picture)#saves Pictures 
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(') #writes an error
        print('--------------------')

In [5]:
def YUVtoRGB(img):
    pictureYUV = cv2.cvtColor(img, cv2.COLOR_YUV2RGB, cv2.IMREAD_UNCHANGED) #uses the CV2 method to convert the color space from YU-V to RGB
    return pictureYUV

In [6]:
def RGBtoYUV(img):
    pictureYUV = cv2.cvtColor(img, cv2.COLOR_RGB2YUV, cv2.IMREAD_UNCHANGED) #uses the CV2 method to convert the color space from RGB to YU-V
    return pictureYUV

In [7]:
def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min() # searches for the smalest number in the img array and saves it in imin
    imax = img.max() # searches for the biggest number in the img array and saves it in imax

    a = (target_type_max - target_type_min) / (imax - imin) # creates ratio of wanted to actual number value space
    b = target_type_max - a * imax # Creates the maximal possible value in b
    try:
        new_img = (a * img + b).astype(target_type) # recalculates the image with the calculated values and sets the new type
    except:
        print('error while converting the image')
    return new_img

In [8]:
def hdrorpng(extention,yuvPic):
    if (extention == 'hdr'): # when hdr than normalize in values between 0 and 1
        print('HDR')
        yuvPic = convert(yuvPic, 0, 1, np.float32) # send to convert in float 32
        #print(yuvPic)
    if (extention == 'png'): # when hdr than normalize in values between 0 and 255
        yuvPic = convert(yuvPic, 0, 255, np.uint8) # normalisation to unit8
    return yuvPic
    

In [9]:
start_time = time.time() #start Timer
#toDo Add parser with args
###### Imput section
path = input('Path to Pictures who should be converted defaut: /home/nico/rohdaten/reallive/png/: ') or '/home/nico/rohdaten/reallive/png/'
inputextention = input('What fileextention do the to read pictures have? [default: png]') or 'png'
outputextention = input('Please type Outputextention[default is: hdr]: ') or 'hdr'
outputpath = input('where to write the stiched pictures to? [default:/home/nico/rohdaten/klein/png/]: ') or '/home/nico/rohdaten/klein/png/'
namePic = input('what should be the Name of the stiched Pictures? [default: pred]') or 'pred'
wantYUV = input('do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]') or 'no'
aOp = sum(1 for f in os.listdir(path) if f.endswith('.'+inputextention)) #summ all ending with extention

Path to Pictures who should be converted defaut: /home/nico/rohdaten/reallive/png/: 
What fileextention do the to read pictures have? [default: png]
Please type Outputextention[default is: hdr]: png
where to write the stiched pictures to? [default:/home/nico/rohdaten/klein/png/]: 
what should be the Name of the stiched Pictures? [default: pred]
do you want to keep YUV color space type y? [deflaut: no (RGB_color space)]


In [28]:
####This file creates Y.png U.png and V.png cromagan pictures
#TO DO normal Pic to YUV einzelbildern
aOp = sum(1 for f in os.listdir(path) if f.endswith('.'+inputextention)) #summ all ending with extention
i = 0 # start at the 0th element in the file system
#print(aOp)
while (i <= aOp-1 ): # read y
    name = os.listdir(path)[i] #finding the name at the current point
    picpath = path + name #combining filename and path
    print(picpath) # prints the path and filename
    pic = readPicture(picpath) # imports the Picture and saves it in pic
    print('waring, Lossy RGB to YU-V conversion')
    yuvPic = RGBtoYUV(pic).astype(np.uint8) # converts the picture to YUV
    u = (yuvPic[:,:,2]) # rearanges the color channels
    v = (yuvPic[:,:,1]) 
    y = (yuvPic[:,:,0])
    savePic(u,(str(i)+'-u_'+namePic),outputextention,outputpath)#saves final U singel channel Picture
    savePic(v,(str(i)+'-v_'+namePic),outputextention,outputpath)#saves final v singel channel Picture
    savePic(y,(str(i)+'-y_'+namePic),outputextention,outputpath)#saves final Y singel channel Picture
    i = i + 1
    
    

/home/nico/rohdaten/reallive/png/2537-u_pred.png
(160, 160)
waring, Lossy RGB to YU-V conversion


error: OpenCV(4.2.0) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<3, 4>; VDcn = cv::impl::{anonymous}::Set<3>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = (cv::impl::<unnamed>::SizePolicy)2u; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1


In [11]:
#Working Version 24012021

print('Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png')
#TO DO Parser
#desc ='yuv to RGB'
#parser = argparse.ArgumentParser(description=desc)
#parser.add_argument('--yuv', type=str, default='./', help='path to Folder of yuv images')
#print(parser.parse_args())
#what, b = parser.parse_known_args()

#if what.yuv == './' :
 #   print('yes')
start_time = time.time()
i = 0
print(aOp)
while (i <= aOp-1 ): # read y
    if ((str(os.listdir(path)[i]).split('-')[1]).split('_')[0]) == 'y': # only searching for y picitures
        name = os.listdir(path)[i] #finding the Name
        print(name +' should be the Y')
        picpath = path + name #combining Name and path
        picy = readPicture(picpath) #reads a pic y to find the x,y axes should be the same for all pictures
        #print(picpath)
        yuvPic = np.zeros((int(picy.shape[0]),int(picy.shape[1]),3)) # generates the x and y achses and channels of picture
        #yuvPic = (yuvPic+1)* 255 # makes all the pixels Black for testing
        yuvPic[:,:,0] = picy # packs the Y in pos 1
        num = (str(os.listdir(path)[i]).split('-')[0])#.split('_')[0]
        newPic = path + num + '-u_'+name.split('_')[1]
        picu = readPicture(newPic)#reads a pic u
        #print(newPic+'should b u')
        #savePic(((picy+1)*255),str(i)+'Nbild42','png','/home/nico/rohdaten/klein/png/')
        yuvPic[:,:,2] = picu # packs the u in pos 0
        #print(yuvPic[:,:,1])
        newPic = path + num + '-v_'+name.split('_')[1] #Generates the Name for v
        #print(newPic+'should beV')
        picv = readPicture(newPic)#reads a pic v
        yuvPic[:,:,1] = picv # packs the u in pos 2
        #Right for the Testdata ist: 0y2u1v <<<<<<<<<<<<<
        #print(yuvPic)
        #TO DO choose if output is RGB oder YUV
        if(wantYUV != 'y'):
            yuvPic = hdrorpng(outputextention,yuvPic) # normalize after conversion
            rgbPic = YUVtoRGB(yuvPic) # to YUV to RGB conversion Matrix needs to have the Format Y[0] U[1] V[2]
            savePic(rgbPic,(str(i)+namePic),outputextention,outputpath)#saves final RGB pic
        if(wantYUV == 'y'):
            yuvPic = hdrorpng(outputextention,yuvPic) # normalize after conversion
            savePic(yuvPic,(str(i)+namePic),outputextention,outputpath)#saves final YUV pic
    i = i + 1

print("--- %s seconds ---" % (time.time() - start_time))
print(str((time.time() - start_time)/60))
print('------------------------- Done --------------------')
#if (path.split('.')[1] == str(extention)): 
#    yuvPic[:,:,0] = picturearray

Pictures in the folder need to have the format: [number]-[y]or[u]or[v].png e.g. : 28-y_pred.png,28-u_pred.png,28-v_pred.png
96
2529-y_pred.png should be the Y
(160, 160)
(160, 160)
(160, 160)
/home/nico/rohdaten/klein/png/1pred.png
2546-y_pred.png should be the Y
(160, 160)
(160, 160)
(160, 160)
/home/nico/rohdaten/klein/png/2pred.png
2531-y_pred.png should be the Y
(160, 160)
(160, 160)
(160, 160)
/home/nico/rohdaten/klein/png/3pred.png
2538-y_pred.png should be the Y
(160, 160)
(160, 160)
(160, 160)
/home/nico/rohdaten/klein/png/6pred.png
2535-y_pred.png should be the Y
(160, 160)
(160, 160)
(160, 160)
/home/nico/rohdaten/klein/png/8pred.png
2534-y_pred.png should be the Y
(160, 160)
(160, 160)
(160, 160)
/home/nico/rohdaten/klein/png/10pred.png
2548-y_pred.png should be the Y
(160, 160)
(160, 160)
(160, 160)
/home/nico/rohdaten/klein/png/17pred.png
2545-y_pred.png should be the Y
(160, 160)
(160, 160)
(160, 160)
/home/nico/rohdaten/klein/png/18pred.png
2532-y_pred.png should be the 

IndexError: list index out of range

In [68]:
start_time = time.time()
print(start_time)
i = 0
while i<=3554343:
    i = i +0.001
print(str((time.time()-start_time)/60))

1611494550.6719644
6.0367039203643795
